# Dependancies

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os, sys, random, argparse, time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
#No module named keras OR cannot import name 'np_utils' if tensorflow.keras
#from keras.utils import np_utils
#from keras.models import load_model
#from keras.models import model_from_json

# Variables

**Hyper-perameters** 

In [2]:
""" Parameters are based off of the 3 layer model in 
    Recurrent Neural Networks for Modeling Motion Capture Data 
    by Mir Khan, Heikki Huttunen, Olli Suominen and Atanas Gotchev
"""

optimizer = keras.optimizers.RMSprop(learning_rate=0.001) # Maintain a moving (discounted) average of the square of gradients
# The folowing initializers are applied to all hidden layers of the model before training 
weight_initializer = keras.initializers.Orthogonal(gain=1.0, seed=None) # Generates an orthogonal matrix with multiplicative factor equal to the gain
recurrent_initializer = tf.keras.initializers.GlorotNormal(seed=None) # Draws samples from a truncated normal distribution centered on 0 with stddev = sqrt(2 / (n_input_weight_units + n_output_weight_units))
bias_initializer = keras.initializers.Zeros() # Set biases to zero
layer_activation = 'tanh'
recurrent_activation = 'hard_sigmoid'
output_activation = 'linear'

batch_size = 32 #number of samples trained before performing one step of gradient update for the loss function (default is stochastic gradient descent)
look_back = 50 #how many frames will be used as a history for prediction
offset = 1 #how many frames in the future is the prediction going to occur at
forecast = 1 #how many frames will be predicted
sample_increment = 25 #number of frames between each sample
epochs = 30 #maximum number of times all training samples are fed into the model for training
units = 1024 #number of nodes in each hidden layer of the network

n_features = 165 #number of columns in the input dimension
frames = 500 #number of frames the model should generate
training_split = .7 #the proportion of the data to use for training
validation_split = .2 #the proportion of the data to use for validating during the training phase at the end of each epoch
evaluation_split = .1 #(test_split) the proportion of the data for evaluating the model effectiveness after training completion

**Variables**

In [33]:
n_features = 165 #number of columns in the input dimension
frames = 500 #number of frames the model should generate
training_split = .7 #the proportion of the data to use for training
validation_split = .2 #the proportion of the data to use for validating during the training phase at the end of each epoch
evaluation_split = .1 #(test_split) the proportion of the data for evaluating the model effectiveness after training completion

In [28]:
csv_data_dir = "/Akamai/MLDance/data/CSV/Raw" #directory to the csv representation of the dances
np_data_dir = "/Akamai/MLDance/data/Numpy" #directory to the numpy representation of the dances
logs_dir = "/Akamai/MLDance/logs" #general output directory

data_identifier = "ts-{}_o-{}_f-{}".format(look_back, offset,forecast) #data-specific string for use in creating readily identifiable filenames
model_identifier = "units-{}_timesteps-{}".format(units, look_back) #model-specific string for use in creating readily identifiable filenames
save_dir = os.path.join(logs_dir, model_identifier) #output directory for model-specific content

Run below block in Jupyter Notebook

In [26]:
class Args():
    def __init__(self):
        self.train = True
args = Args()

Do NOT run this in Jupyter Notebook

In [6]:
parser = argparse.ArgumentParser()

#store_true: default is False, sets the value to True if the respective tag is called
#store_false: default is True, sets the value to False if the respective tag is called
parser.add_argument('--train', action="store_true",
                   help='True: Train on dataset, False: Sample with trained model')

args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [-train]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/mmaddox/.local/share/jupyter/runtime/kernel-66fb3281-02d4-45cb-826e-6a7123f47751.json


SystemExit: 2

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Helper Functions

**General**

In [15]:
def progressbar(it, prefix="", size=60, file=sys.stdout):
    """ Create a visualization of a progress bar updates according to completion status

    :param it: job you are trying to create a progress bar for
    :type obj (sequence or collection)
    :param prefix: The text to display to the left of the status bar
    :type str
    :param size: total length of the progress bar
    :type int
    :param file: what to display/write the progress bar to
    :type output stream
    :return: job you are trying to create a progress bar for
    :rtype: obj (sequence or collection)
    """
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    file.write("\n")
    file.flush()
    
def create_dir(path):
    """ Create the cooresponding directory files for the given path if it does not yet exist

    :param path: proposed directory filepath
    :type str
    :return: created directory filepath
    :rtype: str
    """
    if not os.path.exists(path):
        os.makedirs(path)
    return path

def getFileNames():
    """ Aggregate the names of unique dances

    :return: the dance names where there are csv files for
    :rtype: list
    """
    filenames = [f for f in os.listdir(csv_data_dir) if f.endswith('.csv')]
    for file in enumerate(filenames): #enumerating creates an array where 0 corresponds to the index of the file in filenames and 1 corresponds to the filename
        filenames[file[0]] = '_'.join(file[1].split("_")[:-1])
    return list(set(filenames))

## Data Related

**Helper Functions**

In [35]:
def pre_process_pos_data(pos_data):
    """ Alter the position data such that the X-Y plane movement is relative and the Z remains global
    
    :param pos_data: Hips.X, Hips.Y, and Hips.Z position data
    :type pandas.DataFrame
    :return: dataframe that contains the altered hip positions
    :rtype: pandas.DataFrame
    """
    vertical_movement = pos_data.pop('Hips.Z')
    pos_data = relativize_data(pos_data)
    pos_data['Hips.Z'] = vertical_movement
    return pos_data
    

def relativize_data(data):
    """ Alter the position data such that the X-Y plane movement is relative to the previous frame whilst the Z remains global
    
    :param data: Hips.X and Hips.Y position data for a particular dance
    :type pandas.DataFrame
    :return: dataframe that contains the hip positions
    :rtype: pandas.DataFrame
    """
    next_frame = data.iloc[len(data)-2] #second to last frame
    for index, row in data.iloc[::-1].iterrows(): #itterating through reverse frame order
        if(index == 0):
            data.iloc[index] = 0
        else:
            data.iloc[index] = row - data.iloc[index-1]
    return data

def split_data(data):
    """ Separates the data into three different datasets (training, validation, and evaluation) based off the pre-defined split proportions.
        Each consecutive dataset is selected from the last samples of the previous one.
        
        Data is NOT randomly shuffled before spliting to ensure...
            chopping the data into windows of consecutive samples is still possible
            validation/test results are more realistic, being evaluated on data collected after the model was trained

    :param data: the rot or pos data for a particular dance 
    :type pandas.DataFrame
    :return: the spliced datasets cooresponding to training, validation, and evaluation, respectively
    :rtype: tuple
    """
    train_index = int(len(data)*training_split)
    validation_index = int(len(data)*(training_split+validation_split))
    train_data = data.copy().iloc[0:train_index]
    validate_data = data.copy().iloc[train_index:validation_index]
    evaluate_data = data.copy().iloc[validation_index:]
    return train_data, validate_data, evaluate_data
    
def standardize_data(data, train_mean, train_std):
    """ Standardizes the dataset values by mean subtraction and division by the standard deviation along each dimension
    
    :param data: the rot data for a particular dance 
    :type float
    :param train_mean: mean values over for column
    :type pandas.DataFrame
    :param train_std: standard deviation values for each column
    :type float
    :return: the standardized dataset
    :rtype: pandas.DataFrame
    """
    #standardize rotation (center the values around zero)
    return (data - train_mean) / train_std 

**Pre-Process Wrapper Functions**

In [17]:
def pre_process_data(filename):
    """ Alter the position data such that the X-Y plane movement is relative and the Z remains global
    
    :param filename: Hips.X, Hips.Y, and Hips.Z position data
    :type str
    :return: dataframe that contains the processed dance frames
    :rtype: pandas.DataFrame
    """
    filename = os.path.join(csv_data_dir, filename)
    
    pos_data = pd.read_csv(filename+"_worldpos.csv", usecols=['Hips.X','Hips.Y','Hips.Z'])
    rot_data = pd.read_csv(filename+"_rotations.csv")
    data = rot_data.copy()
    
    # Remove the all the columns were it's all zeroed (End ones)
    zeroed_columns = [column for column in data.columns if 'End' in column]
    for column in zeroed_columns:
        data.pop(column)

    # Standardize rotation (force values from -1 to 1)
    data = data/180.0
    # Standardize rotation (center the values around zero)
    '''standarize = lambda x: (x-x.mean()) / x.std()
    data = data.pipe(standarize)'''
    
    
    '''standardization_json = "standardization_{}-{}-{}.json".format(training_split, validation_split, evaluation_split)
    train_mean = 0
    train_std = 1
    
    with open(standardization_json) as file:
        standardization = json.load(file)
        if filename not in standardization:
            train_data = data.iloc[0:int(len(data)*training_split)]
            standardization.update({filename:{'mean':train_data.mean().to_dict(),'std':train_data.std().to_dict()}})
        train_mean = standardization[filename]['mean']
        train_std = standardization[filename]['std']
        file.close()          
        
    #standardize rotation (center the values around zero)
    data = standardize_data(data, train_mean, train_std)'''

    # Pre-process the position data
    pos_data = pre_process_pos_data(pos_data)
    
        
    # Add the root (hip) data for spacial movement
    data['Hips.Pos.X'] = pos_data.pop('Hips.X')
    data['Hips.Pos.Y'] = pos_data.pop('Hips.Y')
    data['Hips.Pos.Z'] = pos_data.pop('Hips.Z')

    # Remove the time variable from the dataset
    time = data.pop('Time') #maybe change to time change value instead? To indicate speed
    return data

def post_process_data(rotation_df, position_df):
    """ Un-process the data to transform the values representign the generated dance into something MotionBuidler (visualization program) can interpret.
        Un-standardize and un-realativaize the generated dance

    :param data: the processed rot and pos data for the generated dance 
    :type pandas.DataFrame
    :return: the unprocessed versions of the rotation and position frames from the generated dance
    :rtype: tuple
    """
    #undo the normalization and standardization of the data
    rotation_df = rotation_df*180
    position_df['Hips.X'] = position_df['Hips.X'] + hierarchy['offset.x'][0]
    position_df['Hips.Y'] = position_df['Hips.Y'] + hierarchy['offset.y'][0]
    position_df['Hips.Z'] = position_df['Hips.Z'] + hierarchy['offset.z'][0]
    
    new_headers = []
    joints = [j for j in hierarchy['joint'].to_numpy() if "End" not in j]
    for j in joints:
        new_headers.append(j+".Z")
        new_headers.append(j+".X")
        new_headers.append(j+".Y")

    rotation_df = rotation_df.reindex(columns=new_headers)  
    
    rotation_df.insert(0, 'time', np.arange(0.0, 0.03333333*len(rotation_df), 0.03333333))
    position_df.insert(0, 'time', np.arange(0.0, 0.03333333*len(position_df), 0.03333333))
    return rotation_df, position_df

**Save/Load Functions**

In [18]:
def get_processed_data(filename):
    """ Fetch the pre-procced data cooresponding to the given dance name

    :param filename: the name of the dance to grab the data from
    :type str
    :return: the pre-processed dance data
    :rtype: numpy.Array
    """
    #Establish filenames (X is for input, Y is for expected output)
    loaded = os.path.join(np_data_dir, filename)+"_{}_{}_{}".format(training_split, validation_split, evaluation_split)
    
    #If the corresponding numpy file doesn't yet exist, create and save it
    if not (os.path.exists(loaded+".npy")):
        #Print statement for status update
        print("Creating pre-processed datafile:", filename)
        #load the csv file and establish the number of rows and columns
        data = pre_process_data(filename)
        #data = data.iloc[:].values #Enables selection/edit of cells in the pandas dataframe
        np.save(loaded, data)
        print("Saved the pre-processed data to\n\t", loaded)

    return np.load(loaded+".npy")
    
def save_generated_data(generated_data, original_filename, save_filename):
    """ Save the generated dance to a csv file for bvh converstion later.

    :param generated_data: the name of the dance to grab the data from
    :type numpy.Array
    :param original_filename: dance name the generation seed is from
    :type str
    :param save_filename: the directory and filename to store the generated dance at
    :type str
    """
    rotation = generated_data[:,:162] #get the first 162 columns
    position = generated_data[:,[162, 163, 164]] #get the last 3 columns
    hierarchy = pd.read_csv(os.path.join(csv_data_dir, "hierarchy/"+original_filename.split('_')[0]+"_hierarchy.csv"))
    
    data = pd.read_csv(os.path.join(csv_data_dir, original_file+"_rotations.csv"), nrows=0)
    c_headers = [c for c in data.columns if 'End' not in c ][1:]
    rotation_df = pd.DataFrame(rotation, columns=c_headers)
    position_df = pd.DataFrame(position, columns=c_headers[:3])
    
    rotation_df, position_df = post_process_data(rotation_df, position_df)
    
    rotation_df.to_csv(save_filename+"_rot.csv", index=False)
    position_df.to_csv(save_filename+"_pos.csv", index=False)    

**Sequence the Data (Separate Into Samples) Functions**

In [19]:
def sequence_frames(data, dataX, dataY, i):
    """ Create samples (input X and target Y) from the given data

    :param data: the data to create samples from
    :type numpy.Array
    :param dataX: the array to store the input vector data
    :type numpy.Array
    :param dataY: the array to store the output (target) vectors data
    :type numpy.Array
    """
    # Create an input sample cooresponding to the sequence of {look_baack} frame(s) starting at {i}
    seqIn = data[i: i+look_back] 
    # Create an output sample cooresponding to the sequence of {forcast} frame(s) starting at {offset} frame(s) in the future
    seqOut = data[i+look_back+offset-1 : i+look_back+offset+forecast-1] 
    dataX.append(seqIn)
    dataY.append(seqOut)
    
def sequence_data(filename):
    """ Create samples (input X and target Y) from the given data

    :param filename: the name of the dance to sequence data from
    :type str
    :return: the collection of input X and target Y samples
    :type tuple
    """
    #load the csv file and establish the number of rows and columns
    data = get_processed_data(filename)
    N_ROWS = data.shape[0]
    N_COLOMNS = data.shape[1]

    #data = data.iloc[:].values #Enables selection/edit of cells in the dataset
    dataX = []
    dataY = []
        
    #Generate the sequences
    for i in range(0, N_ROWS - look_back, sample_increment): #range(start, stop, step) 
        sequence_frames(data, dataX, dataY, i)

    #X shape [samples, timesteps, features]
    #Y shape [samples, 1, features]
    X, Y = np.array(dataX), np.array(dataY)

    N_SAMPLES = len(dataX)
    Y = np.reshape(Y, (N_SAMPLES, N_COLOMNS))
    return X, Y

In [20]:
def get_sample_data(filename):
    """ Fetch the pre-sequenced or sampled data from the given dance, or create/save it if it does not yet exist

    :param filename: the name of the dance to sample data from
    :type str
    :return: the collection of input X and target Y samples
    :type tuple
    """    
    #Establish filenames (X is for input, Y is for expected output)
    loadedX = os.path.join(np_data_dir, filename+"X_"+data_identifier)
    loadedY = os.path.join(np_data_dir, filename+"Y_"+data_identifier)
    
    #If the corresponding numpy file doesn't yet exist, create and save it
    if not (os.path.exists(loadedX+".npy") and os.path.exists(loadedY+".npy")):
        #Print statement for status update
        print("Creating the sequenced data:", filename)
        X, Y = sequence_data(filename)
        np.save(loadedX, X)
        np.save(loadedY, Y)
        print("Saved the sequenced data to\n\t", loadedX, "\n\t", loadedY)

    return np.load(loadedX+".npy"), np.load(loadedY+".npy")

# Functions related to the Model

**Set-Up Model**

In [37]:
def establish_model(feature_size):
    """ Establish the architecture (layers and how they are connected*) of the model with freshly initialized state for the weights. 
        There is NO compilation information.

    :param feature_size: the number of features in the input/output vector
    :type int
    :return: the model's architecture
    :type keras.Model
    """
    model = keras.Sequential()
    model.add(keras.layers.LSTM(units, 
                                input_shape = (look_back, feature_size), 
                                #batch_size = batch_size, 
                                activation = layer_activation,
                                recurrent_activation = recurrent_activation,
                                kernel_initializer = weight_initializer,
                                recurrent_initializer = recurrent_initializer,
                                bias_initializer = bias_initializer,
                                return_sequences=True))
    model.add(keras.layers.LSTM(units, 
                                activation = layer_activation,
                                recurrent_activation = recurrent_activation,
                                kernel_initializer = weight_initializer,
                                recurrent_initializer = recurrent_initializer,
                                bias_initializer = bias_initializer,
                                return_sequences=True))
    model.add(keras.layers.LSTM(units, 
                                activation = layer_activation,
                                recurrent_activation = recurrent_activation,
                                kernel_initializer = weight_initializer,
                                recurrent_initializer = recurrent_initializer,
                                bias_initializer = bias_initializer,
                                return_sequences=False))
    model.add(keras.layers.Dense(feature_size, activation=output_activation))
    return model

def compile_model(model):
    """ Compile the given model so that it is ready for training and/or prediction/evaluation

    :param model: the model to compile
    :type keras.Model
    :return: the compiled model
    :type keras.Model
    """
    model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])
    print(model.summary())
    return model

**Save and Load Helper Functions**

In [22]:
def save_architecture(model, identifier):
    """ Save the architecture (layers and how they are connected*). 
        Model can be created with a freshly initialized state for the weights and no compilation information from this savefile

    :param model: the model to save
    :type keras.Model
    :param identifier: unique string for creating readily identifiable filenames based off model specs
    :type str
    """
    json_config = model.to_jason()
    print(json_donfig)
    
def save_weights(model, logs=None):
    """ Save the model weights. Ideal for use during training to create checkpoints.
        Weights can be loaded into a model (ideally the original checkpointed model) to extract the desired weights/layers into the saved mode

    :param model: the model to save the weights from
    :type keras.Model
    :param logs: dictionary containing current model specs
    :type dict
    """
    save_filename = "weights_{}_".format(look_back)+model_identifier+"_loss-{:.2f}_acc-{:.2f}.h5".format(logs["loss"], logs["accuracy"])
    model.save_weights(os.path.join(save_dir, save_filename))
    
def save_trained_model(model, identifier):
    """ Save the entire model. Model can be loaded and restart training right where you left off
        The following are saved:
            weight values
            Model's architecture
            Model's training configuration (what you pass to the .compile() method)
            Optimizer and its state, if any (this allows you to restart training)

    :param model: the model to save
    :type keras.Model
    :param identifier: unique string for creating readily identifiable filenames based off model specs
    :type str
    """
    model.save(os.path.join(save_dir, "model-full_"+identifier+".h5"))
    
def load_architecture(file):
    """ Load the architecture (layers and how they are connected*). 
        Model can be created with a freshly initialized state for the weights.
        There is NO compilation information in this savefile.

    :param file: .json file which holds the model's architecture data
    :type str
    :return: the model's architecture
    :type keras.Model
    """
    return keras.models.model_from_json(file)

def load_trained_model(file):
    """ Load the pre-trained model. Compiled when loaded so training/prediction/evaluation can be restarted right where the model left off. 

    :param file: .h5 file which holds the model's information
    :type str
    :return: the compiled model
    :type keras.Model
    """
    return keras.models.load_model(file, compile=True)


# Train Model

In [23]:
class CustomCallback(keras.callbacks.Callback):
    """ A class to create custom callback options. This overrides a set of methods called at various stages of training, testing, and predicting. 
        Callbacks are useful to get a view on internal states and statistics of the model during training.
            Callback list can be passed for .fit(), .evaluate(), and .predict() methods
            
        NOT CURRENTLY IN USE
    """
    def on_train_begin(self, logs=None):
        keys = list(logs.keys())
        print("Starting training; got log keys: {}".format(keys))

    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop training; got log keys: {}".format(keys))

    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start testing; got log keys: {}".format(keys))

    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop testing; got log keys: {}".format(keys))

    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start predicting; got log keys: {}".format(keys))

    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop predicting; got log keys: {}".format(keys))

    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: start of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: end of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: start of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))

In [24]:
def train_model(model):
    """ Trains the model with the dance data.
        The History object's History.history attribute is a record of training loss values and metrics values at successive epochs, 
            as well as cooresponding validation values (if applicable).  

    :param model: the model to train
    :type keras.Model
    :return: the class containing the training metric information and the trained model
    :type tuple
    """
    
    dances = getFileNames()
    callbacks_list = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)]#CustomCallback()
    
    for i in range(epochs):
        for dance in dances:
            print("Epoch ",str(i)+"/"+str(epochs)+":", dance)
            X, Y = get_sample_data(dance)
            
            print ("X Shape:", X.shape)
            print ("Y Shape:", Y.shape)
            #train/fit the model
            history = model.fit(X, Y, 
                      batch_size = batch_size, 
                      callbacks=callbacks_list, 
                      shuffle=False, 
                      validation_split=0.2, 
                      epochs=1, 
                      verbose=1)
        save_weights(model, {'loss':keras.metrics.MeanSquaredError().result(), 'accuracy':keras.metrics.Accuracy().result()})
        if(i%3==0):
            save_trained_model(model_identifier+"epoch-{}.h5".format(i))
        random.shuffle(dances)
    
    print("Done Training")
    return history, model

# Sample/Run Model (Make Predictions)

In [25]:
def benchmark(model, n_frames, random=False):
    """ Generate a dance sequence with the given model

    :param model: the model to use for prediction 
    :type keras.Model
    :param n_frames: the number of frames the model should generate
    :type int
    """
    #select random dance for seed
    dances = getFileNames()
    seed_dance_index = random.randint(0, len(dances) - 1)
    dance = get_processed_data(dances[seed_dance_index])
    seed = dance[:N_TIMESTEPS]
    if random:
        #select random frame(s) for seed
        seed_frame_index = random.randint(0, len(dance) - (look_back+1))
        seed = dance[seed_frame_index:seed_frame_index+look_back]
    
    print("Generating dance with seed from", dances[seed_dance_index])
    #for diversity in [0.2, 0.5, 1.0, 1.2]:
    for diversity in [1.0]:
        start_time = time.time()
        generated = seed
        for i in progressbar(range(n_frames),"{} Progress: ".format(diversity)):
            preds = model.predict(np.array([generated[-look_back:]]), verbose=0)[0]
            generated = np.vstack((generated, preds))
        filename = os.path.join(save_dir, "generated_dance_{}-frames_{}-diversity".format(n_frames, diversity))
        save_generated_data(generated, dances[seed_dance_index], filename)
        print("\tTotal Elapsed time (in sec.):", time.time()-start_time)
        print("\tSaved to", save_location)

# Run Script

In [38]:
def main():
    """ Driver function to control what is run and when if this is the main python script being ran.
        As the project was developed in a jupyter notebook, everything is self-contained in the main file.
        Any expansion, however, would be able to use the predefined classes and functions for whatever purpose without running anything.
    """
    save_location = create_dir(save_dir)
    if(args.train):
        start_time = time.time()
        model = establish_model(n_features)
        train_model(compile_model(model))
        print("--- %s hours ---" % ((time.time() - start_time)/3600))
    else:
        #loads the most recent saved model
        filename = [f for f in os.listdir(save_dir) if "model" in f][-1]
        model = load_trained_model(os.path.join(save_location, filename))
        print(model.summary())
        benchmark(frames, model)
        
if __name__ == "__main__":
    main()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 50, 1024)          4874240   
_________________________________________________________________
lstm_10 (LSTM)               (None, 50, 1024)          8392704   
_________________________________________________________________
lstm_11 (LSTM)               (None, 1024)              8392704   
_________________________________________________________________
dense_3 (Dense)              (None, 165)               169125    
Total params: 21,828,773
Trainable params: 21,828,773
Non-trainable params: 0
_________________________________________________________________
None
Epoch  0/30: Theodora_Happy_1
X Shape: (28, 50, 165)
Y Shape: (28, 165)
Train on 22 samples, validate on 6 samples
22/22 [==============================] - 7s 297ms/sample - loss: 24.7703 - accuracy: 0.0000e+00 - val_loss: 25.4479 - va

KeyboardInterrupt: 